In [1]:
#NON FUNZIONA: CON INTRUSIONE CAMPION DEI POMPIERI

#all imports
import carla #the sim library itself
import time # to set a delay after each photo
import numpy as np #in this example to change image representation - re-shaping
import math
import sys
#sys.path.append('C:/CARLA_0.9.14/PythonAPI/carla') # tweak to where you put carla
from agents.navigation.global_route_planner import GlobalRoutePlanner

# connect to the sim 
client = carla.Client('localhost', 2000)

# define speed contstants
PREFERRED_SPEED = 40 # what it says
SPEED_THRESHOLD = 2 #defines when we get close to desired speed so we drop the

# Max steering angle
MAX_STEER_DEGREES = 40

#camera mount offset on the car - you can tweak these to have the car in view or not
CAMERA_POS_Z = 3 
CAMERA_POS_X = -5 

#adding params to display text to image
font = cv2.FONT_HERSHEY_SIMPLEX
# org - defining lines to display telemetry values on the screen
org = (30, 30) # this line will be used to show current speed
org2 = (30, 50) # this line will be used for future steering angle
org3 = (30, 70) # and another line for future telemetry outputs
org4 = (30, 90) # and another line for future telemetry outputs
org3 = (30, 110) # and another line for future telemetry outputs
fontScale = 0.5
# white color
color = (255, 255, 255)
# Line thickness of 2 px
thickness = 1


# maintain speed function
def maintain_speed(s):
    ''' 
    this is a very simple function to maintan desired speed
    s arg is actual current speed
    '''
    if s >= PREFERRED_SPEED:
        return 0
    elif s < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.9 # think of it as % of "full gas"
    else:
        return 0.4 # tweak this if the car is way over or under preferred speed 

#function to subtract 2 vectors
def angle_between(v1, v2):
    return math.degrees(np.arctan2(v1[1], v1[0]) - np.arctan2(v2[1], v2[0]))

# function to get angle between the car and target waypoint
def get_angle(car,wp):
    '''
    this function to find direction to selected waypoint
    '''
    vehicle_pos = car.get_transform()
    car_x = vehicle_pos.location.x
    car_y = vehicle_pos.location.y
    wp_x = wp.transform.location.x
    wp_y = wp.transform.location.y
    
    # vector to waypoint
    x = (wp_x - car_x)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    y = (wp_y - car_y)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    
    #car vector
    car_vector = vehicle_pos.get_forward_vector()
    degrees = angle_between((x,y),(car_vector.x,car_vector.y))

    return degrees

world = client.get_world()
spawn_points = world.get_map().get_spawn_points()
#look for a blueprint of Mini car
vehicle_bp = world.get_blueprint_library().filter('*mini*')

start_point = spawn_points[0]
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)

# create and show the navigation route like in Tutorial 3
point_a = start_point.location #we start at where the car is
sampling_resolution = 1
grp = GlobalRoutePlanner(world.get_map(), sampling_resolution)
# now let' pick the longest possible route
distance = 0
for loc in spawn_points: # we start trying all spawn points 
                            #but we just exclude first at zero index
    cur_route = grp.trace_route(point_a, loc.location)
    if len(cur_route)>distance:
        distance = len(cur_route)
        route = cur_route
#draw the route in sim window - Note it does not get into the camera of the car
for waypoint in route:
    world.debug.draw_string(waypoint[0].transform.location, '^', draw_shadow=False,
        color=carla.Color(r=0, g=0, b=255), life_time=60.0,
        persistent_lines=True)

# let's start a firetruck @ point 116 
# so it could drive straight and knock our hero car off the navigation route
# so we could see if the car will continue

firetruck_bp = world.get_blueprint_library().filter('*fire*')
firetruck_start_point = spawn_points[116]
firetruck = world.try_spawn_actor(firetruck_bp[0], firetruck_start_point)
    
#setting RGB Camera - this follow the approach explained in a Carla video
# link: https://www.youtube.com/watch?v=om8klsBj4rc&t=1184s

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640') # this ratio works in CARLA 9.14 on Windows
camera_bp.set_attribute('image_size_y', '360')

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
#this creates the camera in the sim
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}
# this actually opens a live stream from the camera
camera.listen(lambda image: camera_callback(image,camera_data))

cv2.namedWindow('RGB Camera',cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera',camera_data['image'])

#main loop 
quit = False
curr_wp = 5 #we will be tracking waypoints in the route and switch to next one wen we get close to current one
predicted_angle = 0
while curr_wp<len(route)-1:
    # Carla Tick
    world.tick()
    if cv2.waitKey(1) == ord('q'):
        quit = True
        vehicle.apply_control(carla.VehicleControl(throttle=0,steer=0,brake=1))
        break
    image = camera_data['image']

    # at a time when the Mini gets to a certain waypoint
    # we get the truck going
    # 150 value makes Mini crash into truck
    # 165 makes the truck crash into the Mini
    if curr_wp==166:
        firetruck.apply_control(carla.VehicleControl(throttle=0.2,steer=0))
    
    while curr_wp<len(route) and vehicle.get_transform().location.distance(route[curr_wp][0].transform.location)<5:
        curr_wp +=1 #move to next wp if we are too close
    
    predicted_angle = get_angle(vehicle,route[curr_wp][0])
    image = cv2.putText(image, 'Steering angle: '+str(round(predicted_angle,3)), org, font, fontScale, color, thickness, cv2.LINE_AA)
    v = vehicle.get_velocity()
    speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
    image = cv2.putText(image, 'Speed: '+str(int(speed)), org2, font, fontScale, color, thickness, cv2.LINE_AA)
    image = cv2.putText(image, 'Next wp: '+str(curr_wp), org3, font, fontScale, color, thickness, cv2.LINE_AA)
    estimated_throttle = maintain_speed(speed)
    # extra checks on predicted angle when values close to 360 degrees are returned
    if predicted_angle<-300:
        predicted_angle = predicted_angle+360
    elif predicted_angle > 300:
        predicted_angle = predicted_angle -360
    steer_input = predicted_angle
    # limit steering to max angel, say 40 degrees
    if predicted_angle<-MAX_STEER_DEGREES:
        steer_input = -MAX_STEER_DEGREES
    elif predicted_angle>MAX_STEER_DEGREES:
        steer_input = MAX_STEER_DEGREES
    # conversion from degrees to -1 to +1 input for apply control function
    steer_input = steer_input/75

    vehicle.apply_control(carla.VehicleControl(throttle=estimated_throttle, steer=steer_input))
    cv2.imshow('RGB Camera',image)
    

#clean up
cv2.destroyAllWindows()
camera.stop()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()

NameError: name 'cv2' is not defined

In [1]:
# CODICE AUTOPILOTA CUSTOM, CALCOLA AUTOMATICAMENTE ANGOLO DI STERZATA 
# BONUS: VISUALIZZA LE COORDINATE DEI SEMAFORI
# SCEGLIE RANDOM UN PERCORSO DAL GRP DISEGNANFOLO SULLA MAPPA. FINITO IL PERCORSO CORRENTE INIZIA UNO NUOVO

#all imports
import carla #the sim library itself
import cv2 #to work with images from cameras
import time # towork with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import math
import random
import sys
sys.path.append('D:/carlasimulator/carla/PythonAPI/carla') # tweak to where you put carla
from agents.navigation.global_route_planner import GlobalRoutePlanner

# connect to the sim 
client = carla.Client('localhost', 2000)

# Constants
# define speed contstants

SPEED_THRESHOLD = 2 #defines when we get close to desired speed so we drop the

# angolo massimo di sterzata
MAX_STEER_DEGREES = 40
# This is max actual angle with Mini under steering input=1.0
STEERING_CONVERSION = 75

#posizione telecamera nella finestra cv
CAMERA_POS_Z = 3 
CAMERA_POS_X = -5 

#parametri per mostrare dati su schermo
font = cv2.FONT_HERSHEY_SIMPLEX
# org - defining lines to display telemetry values on the screen
org = (30, 30) # velocità corrente
org2 = (30, 50) # indica il prossimo angolo di sterzata
org3 = (30, 70) # and another line for future telemetry outputs
org4 = (30, 90) # and another line for future telemetry outputs
org3 = (30, 110) # and another line for future telemetry outputs
fontScale = 0.5
# bianco
color = (255, 255, 255)
# spessore di linea 2px
thickness = 1

#lista dei semafori
def detect_traffic_lights(world):
    # Otteniamo tutti gli attori della mappa
    actors_list = world.get_actors()
    
    # Filtriamo solo gli attori che sono semafori
    tf_list = [actor for actor in actors_list if 'traffic_light' in actor.type_id]

    traffic_lights = {}
    for idx, tf in enumerate(tf_list):
        traffic_lights[tf.get_location()] = tf
    
    return traffic_lights

# funzione utilitaria per catturare e visualizzare l'immagine catturata dal sensore 
def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

# funzione di mantenimento velocità
def maintain_speed(s):
    ''' 
    this is a very simple function to maintan desired speed
    s arg is actual current speed
    '''
    if s >= PREFERRED_SPEED:
        return 0
    elif s < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.9 # think of it as % of "full gas"
    else:
        return 0.4 # tweak this if the car is way over or under preferred speed 


# function to get angle between the car and target waypoint
def get_angle(car,wp):
    '''
    this function returns degrees between the car's direction 
    and direction to a selected waypoint
    '''
    vehicle_pos = car.get_transform()
    car_x = vehicle_pos.location.x
    car_y = vehicle_pos.location.y
    wp_x = wp.transform.location.x
    wp_y = wp.transform.location.y
    
    # vector to waypoint
    x = (wp_x - car_x)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    y = (wp_y - car_y)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    
    #RECUPERA LE DIREZIONI RISPETTO AI PUNTI CARDINALI
    car_vector = vehicle_pos.get_forward_vector()
    degrees = math.degrees(np.arctan2(y, x) - np.arctan2(car_vector.y, car_vector.x))
    # extra checks on predicted angle when values close to 360 degrees are returned
    if degrees<-300:
        degrees = degrees + 360
    elif degrees > 300:
        degrees = degrees - 360
    return degrees

def get_proper_angle(car,wp_idx,rte):
    '''
    This function uses simple fuction above to get angle but for current
    waypoint and a few more next waypoints to ensure we have not skipped
    next waypoint so we avoid the car trying to turn back
    '''
    # create a list of angles to next 5 waypoints starting with current
    next_angle_list = []
    for i in range(10):
        if wp_idx + i*3 <len(rte)-1:
            next_angle_list.append(get_angle(car,rte[wp_idx + i*3][0]))
    idx = 0
    while idx<len(next_angle_list)-2 and abs(next_angle_list[idx])>40:
        idx +=1
    return wp_idx+idx*3,next_angle_list[idx]  

#disegna l'intero percorso random attualmente selezionato
def draw_entire_route(route, duration):
    for waypoint in route:
        world.debug.draw_string(waypoint[0].transform.location, '^', draw_shadow=False, color=carla.Color(r=255, g=255, b=255), life_time=duration, persistent_lines=True)

def draw_route(wp, route,seconds=3.0): #DISEGNA SIMBOLI SUL PERCORSO IN BASE ALLA DISTANZA
    #draw the next few points route in sim window - Note it does not
    # get into the camera of the car
    if len(route)-wp <25: # A 25 PUNTI DALLA FINE COLORA I WP DI ROSSO
        draw_colour = carla.Color(r=255, g=0, b=0)
    else:
        draw_colour = carla.Color(r=0, g=255, b=0) #ALTRIMENTI DI VERDE
    for i in range(10): #STAMPA 10 SIMBOLI DI FRONTE AL VEICOLO
        if wp+i<len(route)-2:
            world.debug.draw_string(route[wp+i][0].transform.location, '^', draw_shadow=False,
                color=draw_colour, life_time=seconds,
                persistent_lines=True)
    return None

#RICERCA DI UN PERCORSO RANDOM LUNGO ALMENO 100 WP MEDIANTE IL GLOBAL ROUTE PLANNER
def select_random_route(position,locs):  
    point_a = position.location #POSIZIONE INIZIALE DELL'AUTO OP DI UN GENERICO WP
    sampling_resolution = 1
    grp = GlobalRoutePlanner(world.get_map(), sampling_resolution)
    # cerca un percorso lungo almeno 100 punti
    min_distance = 100
    result_route = None
    route_list = []
    for loc in locs: # we start trying all spawn points 
                                #but we just exclude first at zero index
        cur_route = grp.trace_route(point_a, loc.location)
        if len(cur_route) > min_distance:  #LUNGHEZZA DEL PERCORSO > 100
            route_list.append(cur_route)
    result_route = random.choice(route_list) #PRESO UN PERCORSO RANDOM NE RITORNA ANCHE IL WP FINALE
    end_of_route = result_route[-1][0].transform 
    return result_route,end_of_route

world = client.get_world()

#lista dei semafori attivi nella mappa
semaphores = detect_traffic_lights(world)

spawn_points = world.get_map().get_spawn_points()
#look for a blueprint of Mini car
vehicle_bp = world.get_blueprint_library().filter('*mini*')

#SPAWN DEL VEICOLO NEL PRIMO WP DISPONIBILE DELLA MAPPA
start_point = spawn_points[0]
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)


#SETTAGGIO CAMERA RGB
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640') # this ratio works in CARLA 9.14 on Windows
camera_bp.set_attribute('image_size_y', '360')
camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
#this creates the camera in the sim
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)
image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()
camera_data = {'image': np.zeros((image_h,image_w,4))}
# LAMBDA CHE CATTURA LE IMMAGINI IN TEMPO REALE DA VISUALIZZARE SU OPENCV
camera.listen(lambda image: camera_callback(image,camera_data))
cv2.namedWindow('RGB Camera',cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera',camera_data['image'])


#MAIN LOOP
start_route = start_point #primo SP della mappa
quit = False
while True:
    # getting a random route for the car
    route,start_route = select_random_route(start_route,spawn_points)
    curr_wp = 5 #tiene traccia dei wp e passa a quello successivo se siamo troppo vicini a quello corrente 
    predicted_angle = 0
    PREFERRED_SPEED = 40 # velocità preferita dall'inizio del percorso

    #colora di bianco l'intero percorso per 60 secondi
    draw_entire_route(route, 60)
    
    while curr_wp < len(route) - 1: #while di percorrenza del percorso attualmente selezionato
        # Carla Tick
        world.tick()
        draw_route(curr_wp, route,1) #disegna 10 simboli ^ di fronte al veicolo, il colore dipende dalla distanza dalla fine 
        
        if cv2.waitKey(1) == ord('q'): #se l'utente preme q sulla finestra opencv, termina l'intero programma, quit = true
            quit = True
            vehicle.apply_control(carla.VehicleControl(throttle=0, steer=0, brake=1))
            break
        
        image = camera_data['image']
        if curr_wp >= len(route) - 10: # il percorso termina a 10 wp dalla fine
            PREFERRED_SPEED = 0 # alla fine del percorso si azzera la velocità
            vehicle.apply_control(carla.VehicleControl(throttle=0,steer=0,brake=1))
            break

        #per debug: stampa S rossa su ogni semaforo della mappa
        #for wps in semaphores:
        #    if vehicle.get_transform().location.distance(wps) < 20:
        #        world.debug.draw_string(wps, str(wps), draw_shadow=False, color=carla.Color(r=255, g=0, b=0), life_time=60, persistent_lines=True)
        
        while curr_wp < len(route)-2 and vehicle.get_transform().location.distance(route[curr_wp][0].transform.location) < 5:
            curr_wp +=1 #si muove al prossimo wp

        #calcola l'eventuale angolo di sterzata in base al prossimo curr_wp
        curr_wp, predicted_angle = get_proper_angle(vehicle, curr_wp, route)
        
        image = cv2.putText(image, 'Steering angle: ' + str(round(predicted_angle,1)), org, font, fontScale, color, thickness, cv2.LINE_AA)
        v = vehicle.get_velocity()
        speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
        image = cv2.putText(image, 'Speed: '+str(int(speed)), org3, font, fontScale, color, thickness, cv2.LINE_AA)
        image = cv2.putText(image, 'Next waypoint: '+str(curr_wp), org2, font, fontScale, color, thickness, cv2.LINE_AA)
        
        estimated_throttle = maintain_speed(speed)

        steer_input = predicted_angle
        # forza l'angolo di sterzata a un massimo predefinito, es.: 40 gradi
        if predicted_angle < -MAX_STEER_DEGREES:
            steer_input = -MAX_STEER_DEGREES
        elif predicted_angle > MAX_STEER_DEGREES:
            steer_input = MAX_STEER_DEGREES

        #rileva il semaforo di fronte al veicolo
        if vehicle.is_at_traffic_light():
            tf = vehicle.get_traffic_light()
            world.debug.draw_string(tf.get_location(), str(tf.get_location()), draw_shadow=False, color=carla.Color(r=255, g=0, b=0), life_time=60, persistent_lines=True)
            while tf.get_state() == carla.TrafficLightState.Red:
                  vehicle.apply_control(carla.VehicleControl(throttle=0,steer=0,brake=1))
        
        # normalizzazione angoli di frenata tra valori [-1.1] per la funzione di controllo del veicolo
        vehicle.apply_control(carla.VehicleControl(throttle=estimated_throttle, steer=steer_input/STEERING_CONVERSION))
        cv2.imshow('RGB Camera',image)
    if quit:
        break

#elimina finestra e veicoli 
cv2.destroyAllWindows()
camera.stop()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()

RuntimeError: time-out of 5000ms while waiting for the simulator, make sure the simulator is ready and connected to localhost:2000